<a href="https://colab.research.google.com/github/alecseiterr/safe_city/blob/main/Taranov_Anatoly/Data_preparation/after_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Доподготовка данных после разметки

Часть 1: Загрузка и распаковка файлов из Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import gdown
import zipfile

# Определите идентификатор файла Google Drive
file_id = "16ReVLVA217aPBxxLaIbR2uG_biyilK"

# Определите URL общего файла
url = f"https://drive.google.com/uc?id={file_id}"

# Укажите папку назначения в Google Colab
output_folder = "/content/unzipped_contents"

# Создайте папку назначения
os.makedirs(output_folder, exist_ok=True)

# Загрузите файл
gdown.download(url, output_folder + "/file.zip", quiet=False)

# Распакуйте файл
with zipfile.ZipFile(output_folder + "/file.zip", "r") as zip_ref:
    zip_ref.extractall(output_folder)

# Выведите список содержимого распакованной папки
os.listdir(output_folder)


Часть 2: Ресайз и разделение изображений

In [ ]:
import gdown
import zipfile
import os
from PIL import Image

# Определите пути к папкам с входными и выходными изображениями
input_images_folder = "/content/drive/MyDrive/CDetection/train/images"
output_images_folder = "/content/drive/MyDrive/CDetection/train_1/images_1"

# Функция для ресайза и разделения изображений
def process_images(input_folder, output_folder, output_size=(1920, 1280), segment_size=(640, 640)):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg"):
            image_path = os.path.join(input_folder, filename)
            image = Image.open(image_path)
            image = image.resize(output_size, Image.ANTIALIAS)
            width, height = image.size
            for i in range(3):
                for j in range(2):
                    left = i * segment_size[0]
                    upper = j * segment_size[1]
                    right = left + segment_size[0]
                    lower = upper + segment_size[1]
                    segment = image.crop((left, upper, right, lower))
                    segment.save(os.path.join(output_folder, f"{filename}_{i}{j}.jpg"))

# Обработка и ресайз изображений
process_images(input_images_folder, output_images_folder)

# Обновление меток (предполагая, что метки - это текстовые файлы)
output_labels_folder = "/content/drive/MyDrive/CDetection/train_1/labels_1"
os.makedirs(output_labels_folder, exist_ok=True)
for filename in os.listdir(input_labels_folder):
    if filename.endswith(".txt"):
        label_path = os.path.join(input_labels_folder, filename)
        with open(label_path, "r") as file:
            lines = file.readlines()
        with open(os.path.join(output_labels_folder, filename), "w") as file:
            for line in lines:
                file.write(f"Изменено размер и разделено: {line}")

print("Задача выполнена.")


<ipython-input-7-b14a3059eb06>:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(output_size, Image.ANTIALIAS)


Задача выполнена.


Часть 3: Разделение меток

In [ ]:

import os

# Определите пути к папкам с входными и выходными метками
input_labels_folder = "/content/drive/MyDrive/CDetection/train/labels"
output_labels_folder = "/content/drive/MyDrive/CDetection/train_1/labels_1"

# Функция для разделения файлов меток
def split_labels(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            label_path = os.path.join(input_folder, filename)
            with open(label_path, "r") as file:
                lines = file.readlines()
            for i in range(3):
                for j in range(2):
                    segment_name = f"{i}{j}"
                    segment_label_path = os.path.join(output_folder, f"{filename}_{segment_name}.txt")
                    with open(segment_label_path, "w") as segment_file:
                        for line in lines:
                            segment_file.write(f"Изменено размер и разделено {segment_name}: {line}")

# Разделение и обновление текстовых файлов меток
split_labels(input_labels_folder, output_labels_folder)

print("Задача выполнена.")


Задача выполнена.


Часть 4. Разделение на выборки Train, Val

In [ ]:
import os
import random
import shutil
import yaml

# Пути к директориям с данными
data_dir = 'images_1'  # Директория с изображениями и метками (labels)
output_dir = '/content/data_custom'  # Директория для подготовленных данных

# Создайте директорию для подготовленных данных, если она не существует
os.makedirs(output_dir, exist_ok=True)

# Создайте поддиректории для обучающей и валидационной выборок
train_dir = os.path.join(output_dir, 'train')
valid_dir = os.path.join(output_dir, 'valid')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

# Получите список всех файлов изображений
image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Установите коэффициент разделения (0.7 для обучающей и 0.3 для валидационной выборок)
split_ratio = 0.7

# Вычислите количество изображений для обучающей и валидационной выборок
num_images = len(image_files)
num_train_images = int(num_images * split_ratio)
num_valid_images = num_images - num_train_images

# Случайным образом перемешайте список изображений
random.shuffle(image_files)

# Разделите изображения на обучающую и валидационную выборки
train_images = image_files[:num_train_images]
valid_images = image_files[num_train_images:]

# Копируйте изображения и метки в соответствующие поддиректории
for image_file in train_images:
    image_path = os.path.join(data_dir, image_file)
    label_file = image_file.replace('.jpg', '.txt')
    label_path = os.path.join(data_dir, label_file)
    shutil.copy(image_path, os.path.join(train_dir, image_file))
    shutil.copy(label_path, os.path.join(train_dir, label_file))

for image_file in valid_images:
    image_path = os.path.join(data_dir, image_file)
    label_file = image_file.replace('.jpg', '.txt')
    label_path = os.path.join(data_dir, label_file)
    shutil.copy(image_path, os.path.join(valid_dir, image_file))
    shutil.copy(label_path, os.path.join(valid_dir, label_file))

# Создайте файл data.yaml для указания путей к обучающим и валидационным данным
data_yaml = {
    'train': '/content/data_custom/train',
    'val': '/content/data_custom/valid',
    'nc': 2,  # Количество классов (для двух классов)
    'names': ['class1', 'class2']  # Имена классов
}

# Сохраните data.yaml
data_yaml_path = os.path.join(output_dir, 'data_custom.yaml')
with open(data_yaml_path, 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file)

# Теперь у вас есть подготовленные данные и файл data.yaml для обучения с моделью YOLO.
